w# Keras Multiple Output Solution
Thanks to https://www.pyimagesearch.com/2018/06/04/keras-multiple-outputs-and-multiple-losses/ and https://www.kaggle.com/kmat2019/neural-network-modeling-with-multiple-outputs for the idea on approaching this problem as a multiple output problem.  Though it doesn't seem to be the favored approach for this competition, I feel that there ought to be a good neural network approach.  This kernel tries a multi-layer, dense neural network implemented in Keras.  The advantage of this approach is that it does not seem to be overfitting, which may pay off against the full dataset.

Ways to improve:
*  I'm not a domain expert in the molecular chem field... I strongly suspect that stronger feature engineering would cause this approach to score higher.  
*  Network architecture:  I'm putting a simpler variant forward here with some options commented out.  There are tweaks that could be made to this architecture that will improve the score.  Forcing the model to overfit to gain a better score on the leaderboard does not usually pay off in the end...
*  More epochs.  The more epochs that can be run without overfitting, the better score could be achieved.  My observation is that even after long training epochs, the model seems to still be learning.

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from keras.layers import Dense, Input, Activation
from keras.layers import BatchNormalization, Add, Dropout
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU
import warnings

warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore", category=DeprecationWarning)
warnings.filterwarnings(action="ignore", category=FutureWarning)

Using TensorFlow backend.


In [3]:
MODEL_DIR = "../models/rich_keras"

In [4]:
os.mkdir(MODEL_DIR)

## Neural Network definition

This neural network is many layers.  In the middle we define our outputs for our two Mullikan charges as well as our Dipole Moment.  The final output is the one we care the most about, the Scalar Coupling Constant.

I think that BatchNormalization at each layer seems superior than small amounts of dropouts.  The network seems to not overfit, even in large numbers of training epochs.  If you do wind up seeing some overfitting, then adding the dropout to a couple of layers ought to help a lot.

In [5]:
def create_nn_model(input_shape):
    inp = Input(shape=(input_shape,))
    x = Dense(512)(inp)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.4)(x)
    
    x = Dense(1024)(inp)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.4)(x)
    
    x = Dense(2048)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    
    x = Dense(2048)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.4)(x)
    
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    #x = Dropout(0.4)(x)
    
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.4)(x)
    
    out1 = Dense(2, activation="linear")(x)  # mulliken charge 2
    out2 = Dense(6, activation="linear")(x)  # tensor 6(xx,yy,zz)
    out3 = Dense(12, activation="linear")(x)  # tensor 12(others)
    
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    
    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    
    out = Dense(1, activation="linear")(x)  # scalar_coupling_constant
    model = Model(inputs=inp, outputs=[out, out1, out2, out3])
    return model

## Plot Function
I rely a lot on loss plots to detect when learning has stopped as well as when overfitting begins.

In [6]:
def plot_history(history, label):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss for %s' % label)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    _ = plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

In [7]:
df_train = pd.read_hdf("../data/champs+giba/train_features.hdf")
df_test = pd.read_hdf("../data/champs+giba/test_features.hdf")

df_label = pd.read_hdf("../data/champs+giba/train_label.hdf")

In [8]:
os.listdir("../data/champs+giba/")

['train_label.hdf',
 'train_target_2.hdf',
 '.ipynb_checkpoints',
 'train_target_3.hdf',
 'test_features.hdf',
 'train_target_1.hdf',
 'train_features.hdf']

In [9]:
df_target_1 = pd.read_hdf("../data/champs+giba/train_target_1.hdf")
df_target_2 = pd.read_hdf("../data/champs+giba/train_target_2.hdf")
df_target_3 = pd.read_hdf("../data/champs+giba/train_target_3.hdf")

In [10]:
df_train = pd.concat([df_train, df_label, df_target_1, df_target_2, df_target_3], axis=1)

In [11]:
test_prediction = np.zeros(len(df_test))

## Main Routine

A bunch of stuff happens here.  Pay attention to the callbacks.  I train a different model for each molecule type, which allows for future retraining.  If you have kept your network the same (except for dropout, etc.), and want to retrain for a few more epochs without having to go back to the beginning, then set the retrain flag to False and it will grab the trained models as starting points.

In [ ]:
from datetime import datetime


mol_types = df_train["type"].unique()
cv_score = []
cv_score_total = 0
epoch_n = 300
verbose = 0
batch_size = 2048

# Set to True if we want to train from scratch.  False will reuse saved models as a starting point.
retrain = True


# Set up GPU preferences
config = tf.ConfigProto(device_count={'GPU': 1, 'CPU': 2})
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6
sess = tf.Session(config=config)
K.set_session(sess)

start_time = datetime.now()

# Loop through each molecule type
for mol_type in mol_types:
    model_name_rd = (
        f'{MODEL_DIR}/molecule_model_{mol_type}.hdf5')
    model_name_wrt = (f'{MODEL_DIR}/molecule_model_{mol_type}.hdf5')
    print('Training %s' % mol_type, 'out of', mol_types, '\n')

    df_train_ = df_train[df_train["type"] == mol_type]
    df_test_ = df_test[df_test["type"] == mol_type]

    # Here's our best features.  We think.
    input_features = [
        "x_0", "y_0", "z_0",
        "x_1", "y_1", "z_1",
        "c_x", "c_y", "c_z",
        'x_closest_0', 'y_closest_0', 'z_closest_0',
        'x_closest_1', 'y_closest_1', 'z_closest_1', 
        "distance", "distance_center0","distance_center1", "distance_c0",
        "distance_c1", "distance_f0", "distance_f1", 
        "cos_c0_c1", "cos_f0_f1", "cos_center0_center1", "cos_c0", "cos_c1",
        "cos_f0", "cos_f1", "cos_center0", "cos_center1",
        "atom_n"
    ]

    # Standard Scaler from sklearn does seem to work better here than other Scalers
    input_data = StandardScaler().fit_transform(
        pd.concat([df_train_.loc[:, input_features], df_test_.loc[:, input_features]]))

    target_data = df_train_.loc[:, "scalar_coupling_constant"].values
    target_data_1 = df_train_.loc[:, ["charge_0", "charge_1"]]
    target_data_2 = df_train_.loc[:, [
        "XX_0", "YY_0", "ZZ_0", "XX_1", "YY_1", "ZZ_1"]]
    target_data_3 = df_train_.loc[:, ["YX_0", "ZX_0", "XY_0", "ZY_0",
                                      "XZ_0", "YZ_0", "YX_1", "ZX_1", "XY_1", "ZY_1", "XZ_1", "YZ_1"]]

    # following parameters should be adjusted to control the loss function
    # if all parameters are zero, attractors do not work. (-> simple neural network)
    m1 = 1
    m2 = 4
    m3 = 1
    target_data_1 = m1*(StandardScaler().fit_transform(target_data_1))
    target_data_2 = m2*(StandardScaler().fit_transform(target_data_2))
    target_data_3 = m3*(StandardScaler().fit_transform(target_data_3))

    # Simple split to provide us a validation set to do our CV checks with
    train_index, cv_index = train_test_split(
        np.arange(len(df_train_)), random_state=111, test_size=0.1)

    # Split all our input and targets by train and cv indexes
    train_input = input_data[train_index]
    cv_input = input_data[cv_index]
    train_target = target_data[train_index]
    cv_target = target_data[cv_index]
    train_target_1 = target_data_1[train_index]
    cv_target_1 = target_data_1[cv_index]
    train_target_2 = target_data_2[train_index]
    cv_target_2 = target_data_2[cv_index]
    train_target_3 = target_data_3[train_index]
    cv_target_3 = target_data_3[cv_index]
    test_input = input_data[len(df_train_):, :]

    # Build the Neural Net
    nn_model = create_nn_model(train_input.shape[1])

    # If retrain==False, then we load a previous saved model as a starting point.
    if not retrain:
        nn_model = load_model(model_name_rd)

    nn_model.compile(loss='mae', optimizer=Adam(lr=0.001))  # , metrics=[auc])

    # Callback for Early Stopping... May want to raise the min_delta for small numbers of epochs
    es = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001,
                                 patience=8, verbose=1, mode='auto', restore_best_weights=True)
    # Callback for Reducing the Learning Rate... when the monitor levels out for 'patience' epochs, then the LR is reduced
    rlr = callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.1, patience=7, min_lr=1e-6, mode='auto', verbose=1)
    # Save the best value of the model for future use
    sv_mod = callbacks.ModelCheckpoint(
        model_name_wrt, monitor='val_loss', save_best_only=True, period=1)

    history = nn_model.fit(train_input, [train_target, train_target_1, train_target_2, train_target_3],
                           validation_data=(
                               cv_input, [cv_target, cv_target_1, cv_target_2, cv_target_3]),
                           callbacks=[es, rlr, sv_mod], epochs=epoch_n, batch_size=batch_size, verbose=verbose)

    cv_predict = nn_model.predict(cv_input)
    plot_history(history, mol_type)

    accuracy = np.mean(np.abs(cv_target-cv_predict[0][:, 0]))
    cv_score.append(np.log(accuracy))
    cv_score_total += np.log(accuracy)

    # Predict on the test data set using our trained model
    test_predict = nn_model.predict(test_input)

    # for each molecule type we'll grab the predicted values
    test_prediction[df_test["type"] == mol_type] = test_predict[0][:, 0]
    K.clear_session()

cv_score_total /= len(mol_types)

Training 1JHC out of ['1JHC' '2JHH' '1JHN' '2JHN' '2JHC' '3JHH' '3JHC' '3JHN'] 



W0718 23:26:33.991197 139654122051392 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0718 23:26:33.991913 139654122051392 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0718 23:26:33.994453 139654122051392 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0718 23:26:34.139677 139654122051392 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0718 23:26:34.153573 

## Prepare results for Submission

The total CV score matches Kaggle's score pretty closely.

In [20]:
from datetime import datetime
datetime.now()

def submit(predictions):
    submit = pd.read_csv('../data/raw/sample_submission.csv')
    print(len(submit), len(predictions))
    submit["scalar_coupling_constant"] = predictions
    submit.to_csv(f"../submits/workingkit-{datetime.now()}-{cv_score_total}.csv.zip", index=False)


submit(test_prediction)

print('Total training time: ', datetime.now() - start_time)

i = 0
for mol_type in mol_types:
    print(mol_type, ": cv score is ", cv_score[i])
    i += 1
print("total cv score is", cv_score_total)

2505542 2505542
Total training time:  0:41:49.760125
1JHC : cv score is  1.049739156936274
2JHH : cv score is  -0.9127646321658073
1JHN : cv score is  0.39138541081891826
2JHN : cv score is  -1.1895794203918477
2JHC : cv score is  -0.4723221583690963
3JHH : cv score is  -1.0482714921450385
3JHC : cv score is  -0.4987939800680683
3JHN : cv score is  -1.56905495863202
total cv score is -0.5312077592520859


In [11]:
import keras
print(keras.__version__)

2.2.4
